# Auth

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-shrgnn')

# ROI

In [2]:
from pyproj import Proj, transform

# Define the EPSG code for EPSG:32637 (UTM Zone 37N) and WGS84
epsg_32637 = Proj(init='epsg:32637')
wgs84 = Proj(init='epsg:4326')

# Coordinates in EPSG:32637
x1, y1 = 647700, 5766250
x2, y2 = 651100, 5769600

# Convert EPSG:32637 coordinates to WGS84
lon1, lat1 = transform(epsg_32637, wgs84, x1, y1)
lon2, lat2 = transform(epsg_32637, wgs84, x2, y2)

# Print the result
print("Longitude:", lon1, lon2)
print("Latitude:", lat1, lat2)

region = ee.Geometry.Polygon(
  [[[lon1, lat2],
    [lon1, lat1],
    [lon2, lat1],
    [lon2, lat2]]], None, False)


Longitude: 41.15290303988647 41.20390338244445
Latitude: 52.0271872172089 52.0563687638839


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-2-7ffbf9927e66>:12: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon1, lat1 = transform(epsg_32637, wgs84, x1, y1)
<

# Pixelwise snow/nosnow comparison for sentinel2 GEE collections

## Count pixels

In [3]:
import ee
import folium

region = region

# Load Sentinel-2 surface reflectance imagery
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
MAX_CLOUD_PROBABILITY = 65

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

# Function to compute NDSI
def computeNDSI(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    ndsi2 = img.normalizedDifference(['B3', 'B12']).rename('NDSI2')
    return img.addBands(ndsi).addBands(ndsi2)

def snow_classifier(img):
    NDSI = img.select('NDSI')
    B03 = img.select('B3')
    NDVI = img.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Thresholds
    NDSI_threshold = 0.4
    brightness_threshold = 0.3

    # Conditions for snow classification
    snow_mask_1 = NDSI.gt(NDSI_threshold)
    snow_mask_2 = NDSI.gt(0.42).And(NDVI.subtract(0.1).abs().lte(0.025))
    snow_mask_3 = NDSI.gt(0.3).And(B03.gt(brightness_threshold))

    # Final snow mask
    snow_mask = snow_mask_1.Or(snow_mask_2).Or(snow_mask_3)

    return snow_mask


map = folium.Map(location=[52.03, 41.16], zoom_start=10)


years = (2019,2020,2021,2022,2023,2024)

snow_counts_full = ee.Image.constant(0)
no_snow_counts_full = ee.Image.constant(0)

snow = []
nosnow = []

for j in years:
    # Create a map centered at the region of interest
    # Initialize empty images to store counts for each class
    snow_counts_img = ee.Image.constant(0)
    no_snow_counts_img = ee.Image.constant(0)
    cloud_counts_img = ee.Image.constant(0)


    START_DATE = ee.Date(f'{j-1}-10-20')
    END_DATE = ee.Date(f'{j}-04-20')

    criteria = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE),
            #ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',90)
        )

    s2Sr_filtered = s2Sr.filter(criteria)
    s2Clouds_filtered = s2Clouds.filter(criteria)

    # Join S2 SR with cloud probability dataset to add cloud mask
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr_filtered,
        secondary=s2Clouds_filtered,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )

    # Apply cloud mask and calculate median
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)

    # Compute NDSI
    s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)

    images = s2CloudMaskedWithNDSI.toList(s2CloudMaskedWithNDSI.size())
    img_orig = s2SrWithCloudMask.toList(s2SrWithCloudMask.size())

    # Loop through images
    l = len(images.getInfo()) #for calculation of the last collection member
    for i in range(images.size().getInfo()):
        image = ee.Image(images.get(i))

        # Create masks
        snow_mask = image.select('NDSI').gt(0.44).And(image.select('B3').gt(0.4))
        no_snow_mask = image.select('NDSI').lte(0.44).Or(image.select('B3').lte(0.4))
        na_mask = ee.Image(image.get('cloud_mask')).select('probability').gt(MAX_CLOUD_PROBABILITY)



        # Increment counts for each class
        #snow_counts_full = snow_counts_img.where(snow_mask, snow_counts_full.add(1))
        #no_snow_counts_full = no_snow_counts_img.where(no_snow_mask, no_snow_counts_full.add(1))
        snow_counts_img = snow_counts_img.where(snow_mask, snow_counts_img.add(1))
        no_snow_counts_img = no_snow_counts_img.where(no_snow_mask, no_snow_counts_img.add(1))
        cloud_counts_img = cloud_counts_img.where(na_mask, cloud_counts_img.add(1))

        #snow_mask = snow_classifier(image)
        #no_snow_mask = snow_mask.Not()


        if int(i)>=l-1:
            print(l)
            snow.append(snow_counts_img)
            nosnow.append(no_snow_counts_img)

        # Clip images to the ROI
        #snow_counts_full = snow_counts_full.clip(region)
        #no_snow_counts_full = no_snow_counts_full.clip(region)
        snow_counts_img = snow_counts_img.clip(region)
        no_snow_counts_img = no_snow_counts_img.clip(region)
        cloud_counts_img = cloud_counts_img.clip(region)

    SnowyFrac = snow_counts_img.divide(no_snow_counts_img)
    mapid6 = SnowyFrac.getMapId({'min': 0, 'max': 2, 'palette': ['red', 'orange', 'yellow', 'green', 'cyan']})  # Assuming cloud_counts_img is your Earth Engine Image
    folium.TileLayer(
        tiles=mapid6['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        opacity=0.5,
        name=f'SnowyFrac {j}'
    ).add_to(map)  # Assuming Map is your Folium map instance
'''
SnowyFracFull = snow_counts_full.divide(no_snow_counts_full)

mapid8 = SnowyFracFull.getMapId({'min': 0, 'max': 2, 'palette': ['red', 'orange', 'yellow', 'green', 'cyan']})  # Assuming cloud_counts_img is your Earth Engine Image
folium.TileLayer(
    tiles=mapid8['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    opacity=0.5,
    name=f'median Snowyfrac image'
).add_to(map)  # Assuming Map is your Folium map instance
'''

# Add the ROI polygon to the map
roi_geojson = region.getInfo()
folium.GeoJson(roi_geojson, name='ROI').add_to(map)

# Display the map
map.add_child(folium.LayerControl())
map

52
72
73
71
72
73


## Export Image

In [ ]:
export_params = {
    'image': SnowyFrac,
    'description': 'median_snowy_frac_image_default',  # Specify the file name
    'folder': 'GEE_images',  # Specify the folder in your Google Drive
    'scale': 10,  # Adjust the scale as needed
    'region': region,  # Define the region of interest
    'maxPixels': 1e13  # Specify the maximum number of pixels
}

# Start the export task
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Print the task status
print('Exporting median SnowyFrac image to Google Drive...')


Exporting median SnowyFrac image to Google Drive...
